In [ ]:
%matplotlib inline
import numpy as np

import astropy.units as u
from astropy.time import Time

import matplotlib.pyplot as plt

from lightkurve import LightCurve
from celerite2 import GaussianProcess

from gadfly import SolarOscillatorKernel, GaussianProcess

In [ ]:
n_durations = 8
simulation_durations = np.logspace(-1, 3, n_durations) * u.day
cadence = 1 * u.min

results = []

for duration in simulation_durations:
    n_samples = int(duration / cadence)
    new_times = np.linspace(0, duration.to(u.day).value, n_samples) * u.d + Time(0, format='bkjd')
    kernel = SolarOscillatorKernel()
    gp = GaussianProcess(kernel, t=new_times)
    runtime = %timeit -o -r 1 gp.sample()
    results.append([duration.to(u.d).value, runtime.average])
results = np.array(results)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.loglog(results[:, 0], results[:, 1], 'ko-')
ax.set(
    xlabel="Duration of simulated light curve [days]",
    ylabel="Runtime [ms]", 
    xticklabels=[f"{x:g}" for x in ax.get_xticks()],
    yticklabels=[f"{1000 * y:g}" for y in ax.get_yticks()]
)

for sp in ['right', 'top']:
    ax.spines[sp].set_visible(False)
fig.savefig('plots/runtime.pdf', bbox_inches='tight')

In [ ]:
print(f"{(np.interp(1, results[:, 1], results[:, 0]) * u.d).to(u.year):.2f} of observations can be computed per second")